In [4]:
import requests

In [8]:
service_url="http://ml-pdf-splitter.rnwl-backend-services.local:5006" #Run on development server in test environment (development-test)

In [6]:
service_url="http://localhost:5006"

In [24]:
def load_pdf_as_bytes(file_path):
    with open(file_path, 'rb') as file:
        return file.read()

# Usage example
pdf_path = '../data/lorem_ipsum.pdf'
pdf_bytes = load_pdf_as_bytes(pdf_path)

In [25]:
requests.get(f'{service_url}/status').content

b'{"status":"OK"}'

In [26]:
import json
response = requests.get(f'{service_url}/lambda_status').content
json.loads(response)

{'status': 'ok',
 'message': 'Service is running and can access the PDF extraction API',
 'pdf_extraction_api': 'accessible',
 'test_response': {'stage': 'test',
  'time_taken': 0.06227922439575195,
  'arn_version': 'arn:aws:lambda:eu-west-2:306232495635:function:ExtractTextFromPDF2:test',
  'function_version': '$LATEST',
  'pdf_id': 0,
  'text': 'Test\xa0document\xa0PDF\xa0\n\xa0\nLorem\xa0ipsum\xa0dolor\xa0sit\xa0amet,\xa0consectetur\xa0adipiscing\xa0elit.\xa0Nulla\xa0est\xa0purus,\xa0ultrices\xa0in\xa0porttitor\xa0\nin,\xa0accumsan\xa0non\xa0quam.\xa0Nam\xa0consectetur\xa0porttitor\xa0rhoncus.\xa0Curabitur\xa0eu\xa0est\xa0et\xa0leo\xa0feugiat\xa0\nauctor\xa0vel\xa0quis\xa0lorem.\xa0Ut\xa0et\xa0ligula\xa0dolor,\xa0sit\xa0amet\xa0consequat\xa0lorem.\xa0Aliquam\xa0porta\xa0eros\xa0sed\xa0\nvelit\xa0imperdiet\xa0egestas.\xa0Maecenas\xa0tempus\xa0eros\xa0ut\xa0diam\xa0ullamcorper\xa0id\xa0dictum\xa0libero\xa0\ntempor.\xa0Donec\xa0quis\xa0augue\xa0quis\xa0magna\xa0condimentum\xa0lobortis.

In [27]:
print(f"EXTRACTED TEXT: \n\n{json.loads(response)['test_response']['text']}")

EXTRACTED TEXT: 

Test document PDF 
 
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla est purus, ultrices in porttitor 
in, accumsan non quam. Nam consectetur porttitor rhoncus. Curabitur eu est et leo feugiat 
auctor vel quis lorem. Ut et ligula dolor, sit amet consequat lorem. Aliquam porta eros sed 
velit imperdiet egestas. Maecenas tempus eros ut diam ullamcorper id dictum libero 
tempor. Donec quis augue quis magna condimentum lobortis. Quisque imperdiet ipsum vel 
magna viverra rutrum. Cras viverra molestie urna, vitae vestibulum turpis varius id. 
Vestibulum mollis, arcu iaculis bibendum varius, velit sapien blandit metus, ac posuere lorem 
nulla ac dolor. Maecenas urna elit, tincidunt in dapibus nec, vehicula eu dui. Duis lacinia 
fringilla massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur 
ridiculus mus. Ut consequat ultricies est, non rhoncus mauris congue porta. Vivamus viverra 
suscipit felis eget condimentum. Cum sociis natoque p

In [10]:
from pydantic import BaseModel

class PolicySummaryRequest(BaseModel):
    fileNames: list
    insuranceType: str

test_s3_filenames = [s3_uri.s3_object for s3_uri in test_s3_uris]
policy_summary_request = PolicySummaryRequest(fileNames=test_s3_filenames, insuranceType='car')


response = requests.post(f'{service_url}/extract/', json=policy_summary_request.model_dump())
response.content

b'[{"name":"Audio, navigation or entertainment equipment","value":null,"extractedText":"","fileName":"unknown","insuranceType":"car","promptId":"1.0class_1.0common_1.0template","modelId":"gpt-3.5-turbo-16k","textExtractorId":"$LATEST"},{"name":"Breakdown","value":"False","extractedText":"policy upgrades, such as Breakdown Cover.","fileName":"resultId_0b679ebc-a2d3-45a3-875d-a94239377ec1_Admiral Motor Cover IPID(1).pdf","insuranceType":"car","promptId":"1.0class_1.0common_1.0template","modelId":"gpt-3.5-turbo-16k","textExtractorId":"$LATEST"},{"name":"Driving abroad","value":"90 days","extractedText":"described in your Certificate of Motor Insurance for 90 days when travelling in any country which is a member of the European Union, Norway, Switzerland, Iceland, Andorra and Liechtenstein.","fileName":"resultId_0b679ebc-a2d3-45a3-875d-a94239377ec1_Admiral Motor Cover IPID(1).pdf","insuranceType":"car","promptId":"1.0class_1.0common_1.0template","modelId":"gpt-3.5-turbo-16k","textExtractor

In [ ]:
import json
json.loads(response.content)

[{'name': 'Breakdown',
  'value': 'False',
  'extractedText': 'policy upgrades, such as Breakdown Cover.',
  'fileName': 'resultId_0b679ebc-a2d3-45a3-875d-a94239377ec1_Admiral Motor Cover IPID(1).pdf',
  'insuranceType': 'car',
  'promptId': '1.0class_1.0common_1.0template',
  'modelId': 'gpt-3.5-turbo-16k',
  'textExtractorId': '$LATEST'},
 {'name': 'Audio, navigation or entertainment equipment',
  'value': None,
  'extractedText': '',
  'fileName': 'unknown',
  'insuranceType': 'car',
  'promptId': '1.0class_1.0common_1.0template',
  'modelId': 'gpt-3.5-turbo-16k',
  'textExtractorId': '$LATEST'},
 {'name': 'Legal assistance',
  'value': '£20,000',
  'extractedText': 'Legal expenses and assistance * £20,000 £25,000 £35,000',
  'fileName': 'resultId_02565818-e282-42f7-a1fc-60d2831eb0fb_S&P IDOL Policy Wording 0922.pdf',
  'insuranceType': 'car',
  'promptId': '1.0class_1.0common_1.0template',
  'modelId': 'gpt-3.5-turbo-16k',
  'textExtractorId': '$LATEST'},
 {'name': 'Personal belong

In [31]:
import requests

def test_extract_text_endpoint(url, pdf_path):
    # Endpoint URL
    endpoint = f"{url}/extract-text/"

    # Open the PDF file in binary mode
    with open(pdf_path, 'rb') as pdf_file:
        # Prepare the files for the request
        files = {'file': ('test.pdf', pdf_file, 'application/pdf')}

        # Send POST request to the endpoint
        response = requests.post(endpoint, files=files)

    # Check the response
    if response.status_code == 200:
        print("Request successful!")
        return response
    else:
        print(f"Request failed with status code: {response.status_code}")
        print("Error message:", response.text)

# Usage
pdf_path = "../data/lorem_ipsum.pdf"  # Adjust this to your PDF file's path

response = test_extract_text_endpoint(service_url, pdf_path)

Request successful!


In [34]:
print(f"EXTRACTED TEXT: \n\n{json.loads(response.content)['text']}")

EXTRACTED TEXT: 

Test document PDF 
 
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla est purus, ultrices in porttitor 
in, accumsan non quam. Nam consectetur porttitor rhoncus. Curabitur eu est et leo feugiat 
auctor vel quis lorem. Ut et ligula dolor, sit amet consequat lorem. Aliquam porta eros sed 
velit imperdiet egestas. Maecenas tempus eros ut diam ullamcorper id dictum libero 
tempor. Donec quis augue quis magna condimentum lobortis. Quisque imperdiet ipsum vel 
magna viverra rutrum. Cras viverra molestie urna, vitae vestibulum turpis varius id. 
Vestibulum mollis, arcu iaculis bibendum varius, velit sapien blandit metus, ac posuere lorem 
nulla ac dolor. Maecenas urna elit, tincidunt in dapibus nec, vehicula eu dui. Duis lacinia 
fringilla massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur 
ridiculus mus. Ut consequat ultricies est, non rhoncus mauris congue porta. Vivamus viverra 
suscipit felis eget condimentum. Cum sociis natoque p